# Healthcare Bluebook Basecamp Project
# L. Clark
# October 2019


Purpose
The purpose of this project is to evaluate your data science skills. We'll be looking for:
Your coding abilities:
Code quality and cleanliness
Documentation
The effectiveness of your solution.
Your thought process.
Your vision for putting your model into production.

The Project
Given the provider data in "Provider_Data.csv", we want to know which of these provider entries represent the same healthcare entity (facility or person). 

For example, the following two Service_Provider_Name entries represent a single healthcare entity:
"Baptist Sleep Centers of South Florida,"
"BAPTIST SLEEP CTR SOUTH FLORIDA"
Please assign a Provider_Entity_ID to each row that can be used to group together providers that represent distinct healthcare entities.

Deliverables:
The following 2 columns in a .csv file named "Output.csv":
Provider_Key: from the "Provider_Data.txt" file
Provider_Entity_ID: a numeric ID that you have generated to group rows into distinct healthcare entities.
All files/code for your work with a simple README file that explains how to run the code.
A 1-2 page summary of your work that answers the following:
Why did you choose the method you chose?
What other methods did you consider and why didn't you use them?
What are the greatest challenges to be overcome in solving this particular problem?
If you had more time (or other resources), what would you do to make your model more effective?
How would you recommend putting your model into production?

Instructions
The due date for this project is 10/18 at 8am. If you need more time due to other commitments, then please let us know as soon as possible.
Use whatever tools and resources you like, but we prefer Python over R. 
Please upload all deliverables to the "Docs & Files" section of Basecamp or provide us a link to a public git repository (GitHub, Bitbucket, GitLab, etc.)
Don't hesitate to ask questions! Email is fine, but our preference would be for you to use the Campfire chat feature of this project.
We expect that this project should take anywhere from 3 to 5 hours of work. You can spend more time, but please don't think we're expecting a comprehensive project with robust documentation. Just show us what you can do.
We know your time is valuable. Thank you for taking on this project and giving us the chance to get to know you better. We've got a $100 Amazon gift card with your name on it as a small way to say thanks. 

Provider Data Format
Provider_Data.txt is a pipe-delimited ("|") flat file with 250,000 rows

Column Definitions
 Provider_Key: a unique key for each entry/row.
 Service_Provider_Name: Name of the service provider as it appeared in the claims data.
 Billing_Provider_Name: Name of the billing provider as it appeared in the claims data.
 Service_Provider_Address_1: Service provider address 1 from the claims data.
 Service_Provider_Address_2: Service provider address 2 from the claims data.
 Service_Provider_City: Service provider city from the claims data.
 Service_Provider_State: Service provider state from the claims data.
 Service_Provider_Zip: Service provider zip code from the claims data.
 Service_Provider_ID: An ID for the provider from the claims data
 Service_Provider_NPI: Service provider NPI from the claims data.
 Member_Market: The CBSA group for the member that received care from the given service provider. From the claims data. Note that the member may be from a different geographic location than the provider.
 Data_Source_ID: An unique id for the vendor that delivered the claims data to healthcare bluebook. (e.g. Cigna, Aetna, etc.)
 Facility_Parent_ID: An id that Healthcare Bluebook has assigned to a specific facility, which is used to map provider data from this data set to that facility.

# Import Libraries

In [15]:
#import python libraries
import pandas as pd
from fuzzywuzzy import fuzz
import numpy as np
from difflib import SequenceMatcher
pd.set_option('display.max_rows', 1000)

# Data Import

In [2]:
#Add the path to provider data file
provider_file_path = '/Users/lindseyclark/Documents/healthcare_bluebook_basecamp_project/Provider_Data.txt'

In [3]:
#read the data to a dataframe
provider_data = pd.read_csv(provider_file_path, delimiter="|")

/Users/lindseyclark/Documents/neural_net_projects/test/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Data Cleaning and Exploratory Data Analysis


In [4]:
#strip whitespace if needed
provider_data = provider_data.applymap(lambda x: x.strip() if type(x)==str else x)

In [13]:
#Explore missing data

In [5]:
#4304 of the rows have a Service_Provider_Name of 'NaN,' and visual inspection 
#of those rows lead me to believe they were useless. Decided to drop--no reasonable way to impute this missing data.
#provider_data.isna().sum()
a = provider_data[provider_data['Service_Provider_Name'].isnull()]
print a
provider_data = provider_data[provider_data['Service_Provider_Name'].notnull()]

        Provider_Key Service_Provider_Name Billing_Provider_Name  \
1118           47016                   NaN                   NaN   
1851           67337                   NaN                   NaN   
4081          125400                   NaN                   NaN   
10839         272151                   NaN                   NaN   
15182         301759                   NaN                   NaN   
34154         683252                   NaN                   NaN   
69027        1148358                   NaN                   NaN   
73786        1399547                   NaN                   NaN   
75139        1677596                   NaN                   NaN   
75172        1660015                   NaN                   NaN   
75382        1694760                   NaN                   NaN   
75426        1708489                   NaN                   NaN   
75437        1699071                   NaN                   NaN   
75477        1664588                   NaN      

In [ ]:
#convert all strings to lower case
#this might be a controversial move--I'm assuming that case is insignificant
provider_data = provider_data.apply(lambda x: x.astype(str).str.lower())

In [218]:
#find nulls or NaNs
provider_data.isna().sum()
provider_data.nunique()


In [124]:
provider_data.isna().sum()

Provider_Key                       0
Service_Provider_Name              0
Billing_Provider_Name         177741
Service_Provider_Address_1     57972
Service_Provider_Address_2    218015
Service_Provider_City           1326
Service_Provider_State          1326
Service_Provider_Zip             457
Service_Provider_ID             6337
Service_Provider_NPI           82235
Member_Market                      0
Data_Source_ID                     0
Facility_Parent_ID            200966
dtype: int64

In [6]:
#based on my exploratory data analysis, I decided to fill the Billing_Provider_Name nulls with the Service_Provider_Name
#values. 
#fill the Billing_Provider_Name NaNs with the values in the Service_provider_Name
provider_data.Billing_Provider_Name.fillna(provider_data.Service_Provider_Name, inplace=True)

In [ ]:
#the Billing_Provider_Name values are what I will match on. 

# Assign Provider_Entity_ID with Fuzzy Matching

In [184]:
#create some useful functions and variables
#string matching function with fuzzywuzzy. Can also use Levenshtein Distance Ratio.

def match_fuzz_string(str2, str1):
    global ratio
    ratio = fuzz.ratio(str1.lower(),str2.lower())
    return ratio

#define a minimum ratio for matching
min_fuzz_ratio = 75

# PRACTICE

In [67]:
#define a function that takes one name and compares it to a list of names
def match_name(name, list_names, min_score=0):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

In [ ]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in missing_salaries.player_name:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, df_salaries.player_name, 75)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"player_name" : name})
    dict_.update({"match_name" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

In [7]:
billing_data = provider_data[['Provider_Key', 'Billing_Provider_Name']].dropna()

In [219]:
provider_data.head()

,Provider_Key,Service_Provider_Name,Billing_Provider_Name,Service_Provider_Address_1,Service_Provider_Address_2,Service_Provider_City,Service_Provider_State,Service_Provider_Zip,Service_Provider_ID,Service_Provider_NPI,Member_Market,Data_Source_ID,Facility_Parent_ID
0,490,Ray O Hatch,NaN,137 East Fort Lowell Road,NaN,Tucson,AZ,85705.0,7350596,NaN,Phoenix-Mesa-Scottsdale AZ,1,NaN
1,893,Neil P. Sheth,NaN,2905 West Warner Road,Suite 12,Chandler,AZ,85224.0,9921276,NaN,Phoenix-Mesa-Scottsdale AZ,1,NaN
2,958,Xenia M. Cabey-Molinar,NaN,1546 North Pkwy. Drive,Suite 101,Gilbert,AZ,85234.0,4253160,NaN,Phoenix-Mesa-Scottsdale AZ,1,NaN
3,1144,Shveta Mehra,NaN,515 North Mesa Drive,NaN,Mesa,AZ,85201.0,7758751,NaN,Phoenix-Mesa-Scottsdale AZ,1,NaN
4,1358,Matthew B. Puleo,NaN,2222 East Highland Avenue,Suite 300,Phoenix,AZ,85016.0,7777854,NaN,Phoenix-Mesa-Scottsdale AZ,1,5.100101e+10


In [11]:
b = billing_data.groupby(['Billing_Provider_Name']).agg(['count'])

In [16]:
b

,Provider_Key
,count
Billing_Provider_Name,
,2
#NAME?,11
& DIABETES CARE CHILDREN ENDOC,2
* * GENERIC VENDOR,1
* HEALTHSOUTH HOLDINGS INC,1
", LLC USACCUSCREEN",1
. - GREENFIELD,1
01367 SOUTHWEST VASCULAR CTR,1


In [12]:
b.sort_values(['count'],ascending=False)

KeyError: 'count'

In [196]:
billing_data_test = billing_data.loc[(billing_data['Billing_Provider_Name'] == "03111 THE LITTLE CLINIC, LLC") |
                                      (billing_data['Billing_Provider_Name'] == "050 PPAZ CHANDLER HEALTH CENTER") |
                                      (billing_data['Billing_Provider_Name'] == "03113 THE LITTLE CLINIC, LLC")]

In [149]:
billing_data_test = billing_data.loc[(billing_data['Billing_Provider_Name'] == "050 PPAZ CHANDLER HEALTH CENTER")]

In [152]:
billing_data_test

,Provider_Key,Billing_Provider_Name
188461,4749325,"03113 THE LITTLE CLINIC, LLC"
191017,4751962,"03113 THE LITTLE CLINIC, LLC"
193630,4756566,"03113 THE LITTLE CLINIC, LLC"
196695,4760872,050 PPAZ CHANDLER HEALTH CENTER
197055,4761524,"03111 THE LITTLE CLINIC, LLC"
197320,4761856,"03113 THE LITTLE CLINIC, LLC"
197586,4762179,"03113 THE LITTLE CLINIC, LLC"
199658,4765983,050 PPAZ CHANDLER HEALTH CENTER
211996,4787237,050 PPAZ CHANDLER HEALTH CENTER
217217,4795203,050 PPAZ CHANDLER HEALTH CENTER


In [188]:
billing_data_test2 = billing_data_test

In [192]:
billing_data_test2.rename(columns={'Billing_Provider_Name': 'Billing_Provider_Name2'}, inplace=True)

/Users/lindseyclark/Documents/neural_net_projects/test/lib/python2.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [193]:
billing_data_test2

,Provider_Key,Billing_Provider_Name2
188461,4749325,"03113 THE LITTLE CLINIC, LLC"
191017,4751962,"03113 THE LITTLE CLINIC, LLC"
193630,4756566,"03113 THE LITTLE CLINIC, LLC"
196695,4760872,050 PPAZ CHANDLER HEALTH CENTER
197055,4761524,"03111 THE LITTLE CLINIC, LLC"
197320,4761856,"03113 THE LITTLE CLINIC, LLC"
197586,4762179,"03113 THE LITTLE CLINIC, LLC"
199658,4765983,050 PPAZ CHANDLER HEALTH CENTER
211996,4787237,050 PPAZ CHANDLER HEALTH CENTER
217217,4795203,050 PPAZ CHANDLER HEALTH CENTER


In [161]:
dict_list = []
provider_id = 0
for Billing_Provider_Name in billing_data_test.Billing_Provider_Name:
    provider_dict = {}
    for key, value in provider_dict:
        match_fuzz_string(Billing_Provider_Name, value)
        if ratio >=75:
            dict_.update({"Billing_Provider_Name" : key})
        else:
            dict_.update({"Billing_Provider_Name" : provider_id + 1})
        provider_id = max(provider_dict, key=provider_dict.get)
        dict_list.append(dict_)
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

""


In [163]:
dict_list = []
provider_id = 0
for Billing_Provider_Name in billing_data_test.Billing_Provider_Name:
    #provider_dict = {}
    match_fuzz_string('03113 THE LITTLE CLINIC, LLC', Billing_Provider_Name)
    print ratio
    #dict_ = {}
    #dict_.update({"Billing_Provider_Name" : Billing_Provider_Name})
    #dict_.update({"Score" : ratio})
    #dict_.update({"score" : match[1]})
    #dict_list.append(dict_)
    
#merge_table = pd.DataFrame(dict_list)
# Display results
#merge_table

NameError: name 'ratio' is not defined

In [186]:
for Billing_Provider_Name in billing_data_test.Billing_Provider_Name:
    match_fuzz_string('03113 THE LITTLE CLINIC, LLC', Billing_Provider_Name)
    print ratio

100
100
100
34
96
100
100
34
34
34


In [199]:
dict_list = []
provider_id = 0
for Billing_Provider_Name in billing_data_test.Billing_Provider_Name:
    for Billing_Provider_Name2 in billing_data_test2.Billing_Provider_Name2:
        match_fuzz_string(Billing_Provider_Name, Billing_Provider_Name2)
        print ratio

100
100
100
34
96
100
100
34
34
34
100
100
100
34
96
100
100
34
34
34
100
100
100
34
96
100
100
34
34
34
34
34
34
100
34
34
34
100
100
100
96
96
96
34
100
96
96
34
34
34
100
100
100
34
96
100
100
34
34
34
100
100
100
34
96
100
100
34
34
34
34
34
34
100
34
34
34
100
100
100
34
34
34
100
34
34
34
100
100
100
34
34
34
100
34
34
34
100
100
100


In [ ]:
result=[]
for sentence in sentence_list:
    if(len(result)==0):
        result.append([sentence])
    else:
        for i in range(0,len(result)):
            score=SequenceMatcher(None,sentence,result[i][0]).ratio()
            if(score<0.5):
                if(i==len(result)-1):
                    result.append([sentence])
            else:
                if(score != 1):
                    result[i].append(sentence)

# old practice

In [87]:
provider_data_test = provider_data.loc[1:1000,]

In [198]:
billing_data_test2

,Provider_Key,Billing_Provider_Name2
188461,4749325,"03113 THE LITTLE CLINIC, LLC"
191017,4751962,"03113 THE LITTLE CLINIC, LLC"
193630,4756566,"03113 THE LITTLE CLINIC, LLC"
196695,4760872,050 PPAZ CHANDLER HEALTH CENTER
197055,4761524,"03111 THE LITTLE CLINIC, LLC"
197320,4761856,"03113 THE LITTLE CLINIC, LLC"
197586,4762179,"03113 THE LITTLE CLINIC, LLC"
199658,4765983,050 PPAZ CHANDLER HEALTH CENTER
211996,4787237,050 PPAZ CHANDLER HEALTH CENTER
217217,4795203,050 PPAZ CHANDLER HEALTH CENTER


In [89]:
provider_data_test_2 = pd.concat([provider_data_test['Provider_Key'], provider_data_test['Service_Provider_Name']], axis=1, )

In [90]:
provider_data_test_2

,Provider_Key,Service_Provider_Name
1,893,Neil P. Sheth
2,958,Xenia M. Cabey-Molinar
3,1144,Shveta Mehra
4,1358,Matthew B. Puleo
5,1644,Stephen S. Shinault
6,1829,Steven M. Lepetich
7,1911,Christopher Aaron Hughes
8,1961,"Spooner Rehab, P.C."
9,1994,Ronald J. Sierzenski
10,2147,DEC Anesthesia LLC


In [91]:
provider_data_test_3 = provider_data_test_2

In [92]:
provider_data_test_3 = provider_data_test_3.rename(columns={'Provider_Key': 'Provider_Key2', 
                                          'Service_Provider_Name': 'Service_Provider_Name_match'})

In [93]:
# List for dicts for easy dataframe creation
dict_list = []
# iterating over our players without salaries found above
for name in provider_data_test_2.Service_Provider_Name:
    # Use our method to find best match, we can set a threshold here
    match = match_name(name, provider_data_test_3.Service_Provider_Name_match, 75)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"Service_Provider_Name" : name})
    dict_.update({"Service_Provider_Name_match" : match[0]})
    #dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

,Service_Provider_Name,Service_Provider_Name_match
0,Neil P. Sheth,Neil P. Sheth
1,Xenia M. Cabey-Molinar,Xenia M. Cabey-Molinar
2,Shveta Mehra,Shveta Mehra
3,Matthew B. Puleo,Matthew B. Puleo
4,Stephen S. Shinault,Stephen S. Shinault
5,Steven M. Lepetich,Steven M. Lepetich
6,Christopher Aaron Hughes,Christopher Aaron Hughes
7,"Spooner Rehab, P.C.","Spooner Rehab, P.C."
8,Ronald J. Sierzenski,Ronald J. Sierzenski
9,DEC Anesthesia LLC,DEC Anesthesia LLC


In [ ]:
#Stuff I'll use


In [120]:
https://stackoverflow.com/questions/40167651/python-iterate-through-a-list-of-strings-and-group-partial-matching-strings

In [68]:
#Practice

In [96]:
'''  
outlist = [(Provider_Key, Service_Provider_Name, Service_Provider_Name)
           for Provider_Key in provider_data_test.Provider_Key
          for Service_Provider_Name in provider_data_test.Service_Provider_Name
          for Service_Provider_Name in provider_data_test.Service_Provider_Name
          ]
print outlist
'''

'  \noutlist = [(Provider_Key, Service_Provider_Name, Service_Provider_Name)\n           for Provider_Key in provider_data_test.Provider_Key\n          for Service_Provider_Name in provider_data_test.Service_Provider_Name\n          for Service_Provider_Name in provider_data_test.Service_Provider_Name\n          ]\nprint outlist\n'

In [94]:
master_list = []
Provider_entity_Id = 1
    for name in provider_data_test_2.Service_Provider_Name:
        # Use our method to find best match, we can set a threshold here
        match = match_name(name, master_list.Service_Provider_Name_match, 75)
        if match[1] >=75:
            
        
        
        # New dict for storing data
        Provider_entity_Id = 1
        dict_ = {}
        dict_.update({"Service_Provider_Name" : name})
        dict_.update({"Provider_entity_Id" : Provider_entity_Id})
        #dict_.update({"score" : match[1]})
        dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table

In [97]:
def copy(dict):
    new_dict = {}
    for key, value in dict:
        if value is a dictionary:
            new_dict[key] = copy(value)
        else if key == 'offset' and value > 50:
            new_dict[key] = 0
        else:
            new_dict[key] = value
    return new_dict

In [ ]:
service_provider_list = []
dict_ = {}
Provider_Entity_Id = 1
for name in provider_data_test_2.Service_Provider_Name:
    Str1 = "united states v. nixon"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
    

In [203]:
from difflib import SequenceMatcher
#sentence_list = ["I love cat", "I love dog", "I love fish", "I hate banana", "I hate apple", "I hate orange"]

result=[]
for sentence in sentence_list:
    if(len(result)==0):
        result.append([sentence])
    else:
        for i in range(0,len(result)):
            score=SequenceMatcher(None,sentence,result[i][0]).ratio()
            if(score<0.5):
                if(i==len(result)-1):
                    result.append([sentence])
            else:
                if(score != 1):
                    result[i].append(sentence)

In [205]:
print(result)

[['03113 THE LITTLE CLINIC, LLC', '03111 THE LITTLE CLINIC, LLC'], ['050 PPAZ CHANDLER HEALTH CENTER'], ['03111 THE LITTLE CLINIC, LLC', '03113 THE LITTLE CLINIC, LLC', '03113 THE LITTLE CLINIC, LLC'], ['050 PPAZ CHANDLER HEALTH CENTER']]


In [319]:
sentence_list = billing_data_test2['Billing_Provider_Name2'].tolist()

In [335]:
result=[]
for sentence in sentence_list:
    if(len(result)==0):
        result.append([sentence])
    else:
        for i in range(0,len(result)):
            score=SequenceMatcher(None,sentence,result[i][0]).ratio()
            #score = match_fuzz_string(sentence, result[i][0])
            if(score<0.75):
                if(i==len(result)-1):
                    result.append([sentence])
            else:
                if(score != 1):
                    result[i].append(sentence)

KeyboardInterrupt: 

In [337]:
len(result)

8515

In [323]:
sentence_list

['Ray O Hatch',
 'Neil P. Sheth',
 'Xenia M. Cabey-Molinar',
 'Shveta Mehra',
 'Matthew B. Puleo',
 'Stephen S. Shinault',
 'Steven M. Lepetich',
 'Christopher Aaron Hughes',
 'Spooner Rehab, P.C.',
 'Ronald J. Sierzenski',
 'DEC Anesthesia LLC',
 'Physiotherapy Associates, Inc.',
 'Timothy D. Kleman',
 'John D. Casalino',
 'Georges Nseir',
 'Clarence R Wall',
 'Manfred H. Ahnen',
 'Sunil Kumar Ram',
 'Nancy H. Kim',
 'Cecil Curtis Graham',
 'Stuart C. Kozinn',
 'Access Medical Laboratories, Inc.',
 'Ernest J. Dorame',
 'Bryan Yau-Wei Wong',
 'Mia Lynne Van Eken',
 'Joshua David Cohen',
 'Quest Diagnostics Incorporated',
 'Curtis Page',
 'Alan Jeffrey Sackin',
 'Sonora Quest Laboratories',
 'Daniel Richard Wright',
 'Medical Diagnostic Laboratories, LLC.',
 'Daniel Corey Grant',
 'Andrea L. DeMets',
 'Robert L. Sawyer',
 'Scott J. Crawford',
 'Banner Desert Medical Center',
 'Pathology Specialists of Arizona',
 'Despina T. Herrera',
 'Bruce R. Kava',
 'Zellman David Skloven',
 'Lyanna 

In [321]:
sentence_list = provider_data['Billing_Provider_Name'].tolist()

In [324]:
a = sentence_list.unique()

AttributeError: 'list' object has no attribute 'unique'

In [291]:
#sentence_list=sentence_list.groupby('Billing_Provider_Name').apply(lambda x: x.reset_index(drop=True)).drop('Billing_Provider_Name',axis=1).reset_index()

In [325]:
type(sentence_list)

list

In [297]:
#sentence_list.drop_duplicates(inplace=True)

In [327]:
#THIS WORKS
sentence_list = provider_data.Billing_Provider_Name.unique()


In [288]:
sentence_list = sentence_list.groupby('Billing_Provider_Name', as_index=False)


In [330]:
type(sentence_list)

numpy.ndarray

In [331]:
len(sentence_list)

125761

In [333]:
sentence_list = sentence_list.tolist()

In [334]:
sentence_list

['Ray O Hatch',
 'Neil P. Sheth',
 'Xenia M. Cabey-Molinar',
 'Shveta Mehra',
 'Matthew B. Puleo',
 'Stephen S. Shinault',
 'Steven M. Lepetich',
 'Christopher Aaron Hughes',
 'Spooner Rehab, P.C.',
 'Ronald J. Sierzenski',
 'DEC Anesthesia LLC',
 'Physiotherapy Associates, Inc.',
 'Timothy D. Kleman',
 'John D. Casalino',
 'Georges Nseir',
 'Clarence R Wall',
 'Manfred H. Ahnen',
 'Sunil Kumar Ram',
 'Nancy H. Kim',
 'Cecil Curtis Graham',
 'Stuart C. Kozinn',
 'Access Medical Laboratories, Inc.',
 'Ernest J. Dorame',
 'Bryan Yau-Wei Wong',
 'Mia Lynne Van Eken',
 'Joshua David Cohen',
 'Quest Diagnostics Incorporated',
 'Curtis Page',
 'Alan Jeffrey Sackin',
 'Sonora Quest Laboratories',
 'Daniel Richard Wright',
 'Medical Diagnostic Laboratories, LLC.',
 'Daniel Corey Grant',
 'Andrea L. DeMets',
 'Robert L. Sawyer',
 'Scott J. Crawford',
 'Banner Desert Medical Center',
 'Pathology Specialists of Arizona',
 'Despina T. Herrera',
 'Bruce R. Kava',
 'Zellman David Skloven',
 'Lyanna 

In [1]:
listtest = [['a', 'b', 'c'], ['d','e', 'f'], ['g', 'h']]

In [2]:
#Step 1 is here. Goal is to create the total number of provider id's and how many times they need to repeat.
#This function will take the list of grouped providers and return 2 more lists, one with the total number of sequential
#Provider_Entity_Ids and the other the a list of the number of repeats.
provider_id = []
repeats = []
def getlists(provider_list):
    for item in provider_list:
        global provider_id
        global repeats
        a = provider_list.index(item)
        provider_id.append(a)
        b = len(item)
        repeats.append(b)
    return provider_id, repeats        

In [3]:
getlists(listtest)

([0, 1, 2], [3, 3, 2])

In [4]:
#Step 2: Get lists of lists of provider ids. Expands the two lists from the function above to get
list_of_provider_entity_ids= []
def get_provider_id_list(ids, vals):
    global list_of_provider_entity_ids
    for i,j in zip(ids, vals):
        a1 = [i]
        a2 = a1*j
        list_of_provider_entity_ids.append(a2)
    return list_of_provider_entity_ids    

In [5]:
get_provider_id_list(provider_id, repeats)

[[0, 0, 0], [1, 1, 1], [2, 2]]

In [ ]:
dataframe_providers = pd.DataFrame(columns=['Provider_entity_Id', 'Billing_Provider_Id'])

In [ ]:
ids = np.arange(1,len(result)+1)

In [ ]:
dataframe = pd.DataFrame(columns=['Provider_Identity_Id','Billing_Provider_Id_Match'])
for item in result:
    df = pd.DataFrame('Provider_Identity_Id' = result[i])
    

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
merged = list(itertools.chain(*list_of_provider_entity_ids))